In [ ]:
import tiktoken


# tiktoken
- tiktoken은 Open AI 모델과 함께 사용할 수 있는 빠른 BPE tokenizer 이다.
---
## 성능
- tiktoken은 비슷한 오픈소스 tokenizer 보다 3~6배 더 빠른 것을 아래 그래프를 통해 확인할 수 있다.
![alt text](perf.svg)

> GPT-2 토크나이저를 사용하여 1GB 텍스트에서 성능 GPT2TokenizerFast을 측정 했습니다 tokenizers==0.13.2.transformers==4.24.0tiktoken==0.2.0



# BPE 이해하기
- BPE는 Byte Pair Encoding 알고리즘

## 전체 흐름
1. 전체 문장을 byte 단위로 하나씩 완전히 분리한다.
    - BPE는 처음부터 문자 단위가 아니라 byte/문자 단위로 시작한다.
    - <This is an ..> -> <T h i s _ i s _ a n ...> 
    - _ : 공백

2. 인접한 문자쌍(pair)를 모두 센다
    - < (T,h), (h,i), (i,s), (s,_), (_,i), ...> 
    - 문장 전체에서 등장한 모든 쌍의 "빈도수"를 센다.

3. 가장 많이 나온 문자쌍을 선택한다.
    - 가장 많이 나온 pair -> <('e', 'n')>
    - 혹은 <(' ', 'e')>
    - 텍스트에 따라 다르지만, **가장 자주 나오는 쌍을 찾는 것이 핵심**

4. 그 쌍을 "한 토큰"으로 병합하고 vocab에 등록한다.
    - <('e', 'n')> -> "en" 이라는 새로운 토큰 생성.


5. 텍스트 안에서 해당 쌍을 "하나의 단위"로 병합하여 다시 token sequence를 만든다.
    - <e n c o d i n g -> (en) c o d i (ng)>

6. 다시 쌍을 카운트하고 → 가장 빈도가 높은 pair를 → 새 토큰으로 병합한다
    - pair 찾기 -> 새 토큰 만들기 -> 텍스트 업데이트

7. vocab_size 개수만큼 계속 반복한다.
    - en
    - ing
    - im
    

In [ ]:
# BPE 이해하기
# BPE는 Byte Pair Encoding 시각화화
from tiktoken._educational import *

# Train a BPE tokeniser on a small amount of text
enc = train_simple_encoding()

# Visualise how the GPT-4 encoder encodes text
enc = SimpleBytePairEncoding.from_tiktoken("cl100k_base")
enc.encode("hello world aaaaaaaaaaaa")

The current most common pair is b' ' + b' '
So we made b'  ' our 257th token
Now the first fifty words in our training data look like:
"""This is an educational implementation of the byte pair encoding algorithm."""

from __future__ import annotations

import collections

import regex

import tiktoken


class SimpleBytePairEncoding:
    def __init__(self, *, pat_str:


The current most common pair is b'  ' + b'  '
So we made b'    ' our 258th token
Now the first fifty words in our training data look like:
"""This is an educational implementation of the byte pair encoding algorithm."""

from __future__ import annotations

import collections

import regex

import tiktoken


class SimpleBytePairEncoding:
    def __init__(self, *, pat_str:


The current most common pair is b'  ' + b' '
So we made b'   ' our 259th token
Now the first fifty words in our training data look like:
"""This is an educational implementation of the byte pair encoding algorithm."""

from __future__ import annotation

[15339, 1917, 264, 70540, 33746]

## Encodings
- Encodings specify how text is converted into tokens. Different models use different encodings.

- tiktoken supports three encodings used by OpenAI models:

### Encoding name/ 	OpenAI models
- o200k_base	: gpt-4o, gpt-4o-mini
- cl100k_base	: gpt-4-turbo, gpt-4, gpt-3.5-turbo, text-embedding-ada-002, text-embedding-3-small, text-embedding-3-large
- p50k_base : Codex models, text-davinci-002, text-davinci-003
- r50k_base : (or gpt2)	GPT-3 models like davinci
You can retrieve the encoding for a model using tiktoken.encoding_for_model() as follows:

In [ ]:
# get_encoding
# 정의된 규칙을 불러오는 메서드 ("cl100k_base")
sentence = "hello world"
enc = tiktoken.get_encoding("cl100k_base")
tokens = enc.encode(sentence)
text = enc.decode(tokens)

print(tokens)
print(text)

[906, 527, 2133, 311, 22963]
we are going to soccer


In [ ]:
# encode() 가 반환된 리스트의 길이(len)를 세면 토큰 개수를 알 수 있다.
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string("tiktoken is great!", "o200k_base")

6

In [ ]:
# 모든 텍스트는 먼저 UTF-8 바이트로 변환된 뒤 BPE 병합 규칙을 적용한다.
# 장점:
# - 어떤 문자든 100% 표현 가능 (이모지, 일본어, 한국어, 혼합 텍스트)
# - 유니코드 깨짐 없음
# - 빠르고 단순함

print([enc.decode_single_token_bytes(token) for token in tokens])

# 이 코드에서는 bytes 를 사람이 읽을 수 있는 문자열로 만들기 위해 GPT 토크나이저는 마지막에 .decode()에서 UTF-8 문자열로 바꾼다.
# 이를 bytes -> str 로 바꾸고 싶다면 .decode() 적용하면된다.

print([enc.decode_single_token_bytes(token).decode('utf-8') for token in tokens])

[b'we', b' are', b' going', b' to', b' soccer']
['we', ' are', ' going', ' to', ' soccer']


In [19]:
# 하나의 문자열을 여러개의 토크나이저로 쪼개는 함수
def compare_encodings(example_string: str) -> None:
    """Prints a comparison of three string encodings."""
    # print the example string
    print(f'\nExample string: "{example_string}"')
    # for each encoding, print the # of tokens, the token integers, and the token bytes
    for encoding_name in ["r50k_base", "p50k_base", "cl100k_base", "o200k_base"]:
        encoding = tiktoken.get_encoding(encoding_name)
        token_integers = encoding.encode(example_string)
        num_tokens = len(token_integers)
        token_bytes = [encoding.decode_single_token_bytes(token) for token in token_integers]
        print()
        print(f"{encoding_name}: {num_tokens} tokens")
        print(f"token integers: {token_integers}")
        print(f"token bytes: {token_bytes}")

In [ ]:
# 같은 단어라도 각각의 토크나이저가 다르게 쪼갠다.
# GPT 계열마다 BPE merge-rule이 다르다.
# 결과를 보면 더 큰 vocab = 더 세밀한 분해 규칙을 가진다.
compare_encodings("antidisestablishmentarianism")


Example string: "antidisestablishmentarianism"

r50k_base: 5 tokens
token integers: [415, 29207, 44390, 3699, 1042]
token bytes: [b'ant', b'idis', b'establishment', b'arian', b'ism']

p50k_base: 5 tokens
token integers: [415, 29207, 44390, 3699, 1042]
token bytes: [b'ant', b'idis', b'establishment', b'arian', b'ism']

cl100k_base: 6 tokens
token integers: [519, 85342, 34500, 479, 8997, 2191]
token bytes: [b'ant', b'idis', b'establish', b'ment', b'arian', b'ism']

o200k_base: 6 tokens
token integers: [493, 129901, 376, 160388, 21203, 2367]
token bytes: [b'ant', b'idis', b'est', b'ablishment', b'arian', b'ism']


In [ ]:
# 모델에 프롬프트를 입력하면 토큰을 어느정도 사용하는지 나타내는 함수
# 이 함수는:

# 메시지 2개

# 각 메시지 오버헤드 3 * 2 = 6

# content 토큰 수 계산

# 마지막 답변 시작 토큰 3개 추가

# → 최종 토큰 수를 계산해준다

def num_tokens_from_messages(messages, model="gpt-4o-mini-2024-07-18"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using o200k_base encoding.")
        encoding = tiktoken.get_encoding("o200k_base")
    if model in {
        "gpt-3.5-turbo-0125",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        "gpt-4o-mini-2024-07-18",
        "gpt-4o-2024-08-06"
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0125.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0125")
    elif "gpt-4o-mini" in model:
        print("Warning: gpt-4o-mini may update over time. Returning num tokens assuming gpt-4o-mini-2024-07-18.")
        return num_tokens_from_messages(messages, model="gpt-4o-mini-2024-07-18")
    elif "gpt-4o" in model:
        print("Warning: gpt-4o and gpt-4o-mini may update over time. Returning num tokens assuming gpt-4o-2024-08-06.")
        return num_tokens_from_messages(messages, model="gpt-4o-2024-08-06")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

In [ ]:

from openai import OpenAI
import os

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))

example_messages = [
    {
        "role": "system",
        "content": "You are a helpful, pattern-following assistant that translates corporate jargon into plain English.",
    },
    {
        "role": "system",
        "name": "example_user",
        "content": "New synergies will help drive top-line growth.",
    },
    {
        "role": "system",
        "name": "example_assistant",
        "content": "Things working well together will increase revenue.",
    },
    {
        "role": "system",
        "name": "example_user",
        "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage.",
    },
    {
        "role": "system",
        "name": "example_assistant",
        "content": "Let's talk later when we're less busy about how to do better.",
    },
    {
        "role": "user",
        "content": "This late pivot means we don't have time to boil the ocean for the client deliverable.",
    },
]

for model in [
    "gpt-3.5-turbo",
    "gpt-4-0613",
    "gpt-4",
    "gpt-4o",
    "gpt-4o-mini"
    ]:
    print(model)
    # example token count from the function defined above
    print(f"{num_tokens_from_messages(example_messages, model)} prompt tokens counted by num_tokens_from_messages().")
    # example token count from the OpenAI API
    response = client.chat.completions.create(model=model,
    messages=example_messages,
    temperature=0,
    max_tokens=1)
    print(f'{response.usage.prompt_tokens} prompt tokens counted by the OpenAI API.')
    print()

gpt-3.5-turbo
129 prompt tokens counted by num_tokens_from_messages().


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: <your Op*******************************var>. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
import sentencepiece as spm

# 학습
spm.SentencePieceTrainer.Train(
    input="corpus.txt",
    model_prefix="spm",
    vocab_size=2000,
)

# 사용
sp = spm.SentencePieceProcessor(model_file="spm.model")
ids = sp.encode("hello world", out_type=int)
tokens = sp.encode("hello world", out_type=str)


---

# 🔥 AutoTokenizer란?

### ✔ “모델 이름만 넣으면 해당 모델에 맞는 토크나이저를 자동으로 불러오는 클래스”

예:

```python
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
```

이 한 줄이면,

* WordPiece 기반 BERT tokenizer
* BERT vocabulary 파일
* special token 설정 ([CLS], [SEP], [PAD], [MASK])
* normalization, pre-tokenization 규칙
  모두 자동으로 로드된다.

📌 **즉, "모델에 딱 맞는 tokenizer를 자동으로 찾아주는 도구"**

---

# 🔥 AutoTokenizer의 내부 동작

`from_pretrained("bert-base-uncased")` →
HuggingFace Hub에서 해당 모델의 파일들을 다운로드:

* vocab.txt
* tokenizer.json
* tokenizer_config.json
* special_tokens_map.json

AutoTokenizer는 이 파일을 읽고 자동으로 올바른 tokenizer 클래스를 선택한다.

예:

* bert → BertTokenizer
* gpt2 → GPT2Tokenizer
* llama → LlamaTokenizer
* t5 → T5Tokenizer

너는 그걸 신경 쓸 필요 없이
**AutoTokenizer가 알아서 선택**한다.

---

# 🔥 tokenizer("Hello world!", return_tensors="pt") 설명

이 코드는 토큰화 + 텐서 변환을 한 번에 한다.

출력 예:

```python
ids = tokenizer("Hello world!", return_tensors="pt")
```

ids 내부에는 4가지 주요 값이 있다.

### 1) input_ids

문장의 토큰 ID 리스트
예:

```
[101, 7592, 2088, 999, 102]
```

### 2) attention_mask

패딩된 부분은 0, 실제 토큰은 1

예:

```
[1, 1, 1, 1, 1]
```

### 3) token_type_ids

BERT에서 문장 구분(Sentence A/B) 용도
단일 문장일 때는 모두 0

### 4) special tokens 포함 여부 (자동)

* [CLS] 앞에 붙음
* [SEP] 끝에 붙음

---

# 🔥 AutoTokenizer 주요 메서드(실전에서 꼭 쓰는 것들)

## 1. `tokenizer.encode()`

문자열을 토큰 ID로 변환

```python
ids = tokenizer.encode("Hello world!")
```

---

## 2. `tokenizer.decode()`

토큰 ID → 문자열 복원

```python
text = tokenizer.decode(ids)
```

---

## 3. `tokenizer.tokenize()`

토큰 문자열 단위 확인하기

```python
tokens = tokenizer.tokenize("Hello world!")
# ['hello', 'world', '!']
```

---

## 4. `tokenizer.batch_encode_plus()`

텍스트 여러 개 한 번에 토큰화
(요즘은 __call__이 대신함)

---

## 5. `tokenizer.add_tokens()`

special token 추가할 때 많이 씀

```python
tokenizer.add_tokens(["<title>", "</title>"])
```

---

## 6. `tokenizer.pad()`

수동 패딩

```python
batch = tokenizer.pad(input_dict, return_tensors="pt")
```

---

## 7. `tokenizer.batch_decode()`

배치 단위 토큰 → 텍스트 복원

---

## 8. `tokenizer.save_pretrained()`

로컬에 tokenizer 저장

```python
tokenizer.save_pretrained("./my_tokenizer")
```

---

# 🔥 추가로 알아야 할 실제 스킬

tokenizer 안의 중요 속성:

```python
tokenizer.vocab_size        # 전체 vocab 크기
tokenizer.model_max_length  # 모델 최대 입력 길이
tokenizer.special_tokens_map
tokenizer.all_special_tokens
tokenizer.pad_token
tokenizer.cls_token
tokenizer.sep_token
tokenizer.unk_token
```

---

# 🔥 AutoTokenizer를 왜 다들 써?

### ✔ 모델마다 tokenizer 구현이 다 다르다

GPT2는 BPE
BERT는 WordPiece
T5는 SentencePiece
Llama는 SentencePiece/BPE 변형

이걸 사람이 일일이 맞추는 건 불가능함.

### ✔ AutoTokenizer는 "모델에 딱 맞는" tokenizer를 자동 선택

→ 실험 과정에서 오류 최소화
→ 같은 모델에서 학습한 토크나이저 그대로 사용 가능

---

# 🔥 예시로 코드에 라벨 추가해놓기

```python
from transformers import AutoTokenizer

# 1. Bert 전용 tokenizer 자동 로딩
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 2. 입력을 텐서로 변환
ids = tokenizer("Hello world!", return_tensors="pt")

print(ids["input_ids"])       # 토큰 ID
print(ids["attention_mask"])  # 마스크
```

---




In [17]:
# Hugging Face transformers의 AutoTokenizer (모델별 토크나이저 래퍼)
# 용도: BERT, GPT-2, LLaMA, T5 등 프리트레인된 모델과 같이 쓰는 토크나이저
# 내부적으로는 tokenizers / WordPiece/ BPE / SentencePiece등을 알아서 사용
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
ids = tokenizer("Hello world!", return_tensors="pt")
# ids["input_ids"], ids["attention_mask"] 등

# 문장의 토큰 id 리스트
print(ids["input_ids"])

# 패딩된 부분은 0, 실제 토큰은 1
print(ids["attention_mask"])

# 단일 문장일 때는 모두 0
print(ids["token_type_ids"])


tensor([[ 101, 7592, 2088,  999,  102]])
tensor([[1, 1, 1, 1, 1]])
tensor([[0, 0, 0, 0, 0]])


In [20]:
# 문자열을 토큰 ID로 변환환
ids = tokenizer.encode("Hello world!")
print(ids)

# 토큰 ID -> 문자열로 복원
text = tokenizer.decode(ids)
print(text)

# 토큰 문자열 단위 확인하기
token = tokenizer.tokenize("Hello world!")
print(token)

# special token 추가하기
print("Before:", tokenizer.vocab_size)
tokenizer.add_tokens(["<title>", "</title>"])
print("After:", tokenizer.vocab_size)

[101, 7592, 2088, 999, 102]
[CLS] hello world! [SEP]
['hello', 'world', '!']
Before: 30522
After: 30522


In [ ]:
# 스페셜 토큰 확인인
tokenizer.get_added_vocab()

{'[PAD]': 0,
 '[UNK]': 100,
 '[CLS]': 101,
 '[SEP]': 102,
 '[MASK]': 103,
 '<title>': 30522,
 '</title>': 30523}

In [ ]:
# 로컬에 tokenizer 저장
tokenizer.save_pretrained("./my_tokenizer")

In [22]:
print(tokenizer.vocab_size)
print(tokenizer.model_max_length)
print(tokenizer.special_tokens_map)
print(tokenizer.all_special_tokens)
print(tokenizer.pad_token)
print(tokenizer.cls_token)
print(tokenizer.sep_token)
print(tokenizer.unk_token)


30522
512
{'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}
['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']
[PAD]
[CLS]
[SEP]
[UNK]
